In [1]:
import pandas as pd

import numpy as np
import os

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import seaborn as sns
from sklearn.metrics import recall_score, precision_score, f1_score 
import pickle

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from collections import Counter  

from feature_engine.discretisation import EqualWidthDiscretiser
from sklearn.tree import DecisionTreeClassifier
import funciones

from feature_engine.discretisation import DecisionTreeDiscretiser

In [2]:
datos = pd.read_csv('/home/juan/proyectoTesis/datasets/creditcard.csv')

datos.shape

#Ver balanceo de la clase objetivo. Diabetes_012
porcentaje = 100*datos['Class'].value_counts()/len(datos)
print(porcentaje.astype(str) + '%')

Class
0       99.827251436938%
1    0.1727485630620034%
Name: count, dtype: object


In [3]:
X = datos.drop('Class', axis=1)
y= datos['Class'] 

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y)

print(y_test.value_counts())



porcentaje = 100 * y_train.value_counts()/len(y_train)

porcentaje

Class
0    71079
1      123
Name: count, dtype: int64


Class
0    99.827251
1     0.172749
Name: count, dtype: float64

In [4]:
columnas_disc = X_train.columns.to_list()

discretizador = DecisionTreeDiscretiser(bin_output="bin_number",
                                        cv=3,
                                        scoring="roc_auc",
                                        variables= columnas_disc, 
                                        regression=False)

discretizador.fit(X_train, y_train)

DecisionTreeDiscretiser(bin_output='bin_number', regression=False,
                        scoring='roc_auc',
                        variables=['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6',
                                   'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13',
                                   'V14', 'V15', 'V16', 'V17', 'V18', 'V19',
                                   'V20', 'V21', 'V22', 'V23', 'V24', 'V25',
                                   'V26', 'V27', 'V28', 'Amount'])

In [5]:
X_train_disc = discretizador.transform(X_train)
X_test_disc = discretizador.transform(X_test)

In [6]:
#Discretizar todos los datos para usar la tasa de inconsistencia

X = datos.drop("Class", axis=1)
y = datos["Class"]

disc = DecisionTreeDiscretiser(bin_output="bin_number",
                                        cv=3,
                                        scoring="roc_auc",
                                        variables= columnas_disc, 
                                        regression=False)

disc.fit(X, y)

DecisionTreeDiscretiser(bin_output='bin_number', regression=False,
                        scoring='roc_auc',
                        variables=['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6',
                                   'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13',
                                   'V14', 'V15', 'V16', 'V17', 'V18', 'V19',
                                   'V20', 'V21', 'V22', 'V23', 'V24', 'V25',
                                   'V26', 'V27', 'V28', 'Amount'])

In [7]:
X_disc = disc.transform(X)

X_disc.join(y)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,10,0,13,1,11,9,13,3,12,...,2,2,7,5,2,0,2,4,11,0
1,0,10,0,13,0,11,9,13,3,12,...,2,2,7,1,2,0,2,4,5,0
2,0,10,0,13,0,11,9,13,3,9,...,2,2,10,1,2,0,2,4,12,0
3,0,10,0,13,0,11,9,13,3,11,...,2,2,7,1,2,0,2,4,11,0
4,0,10,0,13,0,11,9,13,3,12,...,2,2,7,6,2,1,2,4,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,1,5,7,6,0,5,2,8,6,12,...,2,2,10,1,5,0,5,8,1,0
284803,1,10,0,13,0,11,9,13,3,12,...,2,2,7,1,2,0,2,4,7,0
284804,1,11,0,12,0,11,9,13,3,12,...,2,2,7,8,2,0,2,4,8,0
284805,1,10,0,13,0,11,9,13,3,12,...,2,2,7,6,2,1,2,4,7,0


In [8]:
tasa = funciones.tasa_inconsistencia(X_disc.join(y)X_disc.join(y), list(X_disc.columns), "Class")

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2102774140.py, line 1)

In [ ]:
tasa

8.426759173756263e-05

In [ ]:
subconjunto =X_disc.drop(columns=["Time", "Amount", "V15", "V16", "V17"]).columns.to_list()

tasa2=funciones.tasa_inconsistencia(X_disc.join(y), subconjunto, "Class")
tasa2

0.00011586793863914862

In [ ]:
X = datos[subconjunto]
y= datos['Class'] 

X_train, X_test,y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

model_lr_desbalanceado = LogisticRegression(C=1.0,penalty='l2',random_state=1,solver="newton-cg")

model_lr_desbalanceado.fit(X_train, y_train)


LogisticRegression(random_state=1, solver='newton-cg')

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_RndmForest = RandomForestClassifier(random_state=57364)

model_RndmForest.fit(X_train,y_train)

RandomForestClassifier(random_state=57364)